In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Concatenate, RepeatVector
from tensorflow.keras.preprocessing.sequence import pad_sequences
from kerastuner.tuners import BayesianOptimization
from tensorflow.keras.optimizers import Adam, SGD

# 데이터 불러오기
df = pd.read_csv('NVspinData_None_-1_230807.csv')

# 전체 데이터셋에서 시퀀스 중에서 최대 길이 구하기
all_sequences = df['combination'].apply(eval).tolist()
max_seq_length = max([len(seq) for seq in all_sequences])

# 데이터셋 분리
train_df, test_df = train_test_split(df, shuffle=True, test_size=0.2)
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=0.2)

# 훈련, 검증, 테스트 데이터 준비
theta_train = train_df['Theta'].values.reshape(-1, 1)
phi_train = train_df['Phi'].values.reshape(-1, 1)
sequence_train = pad_sequences(train_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_val = val_df['Theta'].values.reshape(-1, 1)
phi_val = val_df['Phi'].values.reshape(-1, 1)
sequence_val = pad_sequences(val_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_test = test_df['Theta'].values.reshape(-1, 1)
phi_test = test_df['Phi'].values.reshape(-1, 1)
sequence_test = pad_sequences(test_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

# 모델 생성 함수
def build_rnn_model(hp):
    # Inputs
    theta_input = Input(shape=(1,), name='theta_input')
    phi_input = Input(shape=(1,), name='phi_input')
    
    merged = Concatenate()([theta_input, phi_input])
    
    repeated_vector = RepeatVector(max_seq_length)(merged)
    
    # SimpleRNN Layer
    rnn_units = hp.Int('rnn_units', min_value=16, max_value=128, step=16)
    rnn_layer = SimpleRNN(units=rnn_units, return_sequences=True)(repeated_vector)

    # Output Layer
    output_layer = Dense(max_seq_length, activation='softmax')(rnn_layer)

    model = Model(inputs=[theta_input, phi_input], outputs=output_layer)

    # Optimizer choice
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd'])
    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3))
    else:
        optimizer = SGD(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

tuner = BayesianOptimization(
    build_rnn_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='my_dir',
    project_name='keras_tuner_bayesian_rnn'
)

tuner.search_space_summary()

max_epochs = 30
# Train the tuner
tuner.search([theta_train, phi_train], sequence_train, 
             validation_data=([theta_val, phi_val], sequence_val),
             epochs=max_epochs, batch_size=32)

# 최적의 하이퍼파라미터를 가진 모델 가져오기
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# 검증
val_predictions = best_model.predict([theta_val, phi_val])
# 임의로 테스트 샘플 5개 확인
indices = np.random.choice(len(theta_test), 5, replace=False)
for i in indices:
    print(f"Theta: {theta_test[i]}, Phi: {phi_test[i]}")
    print("Actual Sequence:", sequence_test[i])
    print("Predicted Sequence:", np.argmax(val_predictions[i], axis=-1))
    print("----")

# 훈련 과정 그래프로 확인
history = best_model.fit([theta_train, phi_train], np.array(sequence_train),
                         validation_data=([theta_val, phi_val], np.array(sequence_val)),
                         epochs=10, batch_size=32)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()


Trial 2 Complete [00h 00m 00s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
64                |16                |rnn_units
adam              |sgd               |optimizer
0.00029273        |0.00016459        |learning_rate

Epoch 1/30


Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "c:\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Python310\lib\site-packages\keras_tuner\engine\hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "c:\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in e